## Milestone 2 Task 3

In [3]:
from scripts.database_utils import DatabaseConnector
from scripts.data_cleaning import DataCleaning
from scripts.data_extraction import DataExtractor

Create our DataFrame which will contain the user data.

In [4]:
# Create an instance of the DatabaseConnector class
database = DatabaseConnector()
# Initialise engine with yaml_file_path set to 'default'
database_engine = database.init_db_engine()
# Create an instance of the DataExtractor class
data_extractor = DataExtractor()
# Extract data from table 'legacy_users' and store it in a DataFrame
df = data_extractor.read_rds_table(database_engine, 'legacy_users')

Called 'init_db_engine' method for database engine initialisation.
Read credentials from db_creds.yaml
Database credentials now stored in 'credentials' variable.
Database engine initialised successfully with the credentials in db_creds.yaml.
Data extracted from the 'legacy_users' table and into a DataFrame.


Now, clean the DataFrame.

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15320 entries, 0 to 15319
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   index          15320 non-null  int64 
 1   first_name     15320 non-null  object
 2   last_name      15320 non-null  object
 3   date_of_birth  15320 non-null  object
 4   company        15320 non-null  object
 5   email_address  15320 non-null  object
 6   address        15320 non-null  object
 7   country        15320 non-null  object
 8   country_code   15320 non-null  object
 9   phone_number   15320 non-null  object
 10  join_date      15320 non-null  object
 11  user_uuid      15320 non-null  object
dtypes: int64(1), object(11)
memory usage: 1.4+ MB


In [6]:
# Create an instance of the DataCleaning class, with our DataFrame passed as an argument
data_cleaner = DataCleaning(df)
# Clean the DataFrame
cleaned_df = data_cleaner.clean_user_data(df)

DataFrame cleaning operation initiated.
DataFrame cleaned successfully.


In [7]:
cleaned_df['country_code'].unique()

array(['DE', 'GB', 'US'], dtype=object)

In [8]:
cleaned_df.info()

# Can you confirm how many user_uuids should be found in the order_table?
# This SQL query SELECT COUNT(DISTINCT user_uuid) FROM orders_table; states that it is 15284.
# So the cleaning operation should result in 15284 non-null values in total?


<class 'pandas.core.frame.DataFrame'>
Index: 15284 entries, 0 to 1249
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   first_name     15284 non-null  object        
 1   last_name      15284 non-null  object        
 2   date_of_birth  15284 non-null  datetime64[ns]
 3   company        15284 non-null  object        
 4   email_address  15284 non-null  object        
 5   address        15284 non-null  object        
 6   country        15284 non-null  object        
 7   country_code   15284 non-null  object        
 8   phone_number   15284 non-null  object        
 9   join_date      15284 non-null  datetime64[ns]
 10  user_uuid      15284 non-null  object        
dtypes: datetime64[ns](2), object(9)
memory usage: 1.4+ MB


In [9]:
# Assuming cleaned_df is your dataframe

# Filter the dataframe to include only rows with at least one NaN value
rows_with_null = cleaned_df[cleaned_df.isna().any(axis=1)]

# Display the resulting rows
print(rows_with_null)


Empty DataFrame
Columns: [first_name, last_name, date_of_birth, company, email_address, address, country, country_code, phone_number, join_date, user_uuid]
Index: []


In [18]:
# Count number of rows in the cleaned_df
# df.info()
# cleaned_df.info()

import pandas as pd

# string_rows = cleaned_df[cleaned_df['join_date'].map(lambda x: isinstance(x, str))] 
# print(string_rows['join_date'])

# # Sort and reverse the unique join dates
# sorted_unique_join_dates = sorted(string_rows['join_date'].unique(), reverse=True)
# print(sorted_unique_join_dates)


duplicate_rows = cleaned_df[cleaned_df.duplicated(subset=['user_uuid'])]
print(duplicate_rows)
# # Assuming df is your DataFrame
# unique_sorted_values = sorted(cleaned_df['user_uuid'].unique())

# # Display the sorted unique values
# print("Sorted Unique Values:", unique_sorted_values)


Empty DataFrame
Columns: [first_name, last_name, date_of_birth, company, email_address, address, country, country_code, phone_number, join_date, user_uuid]
Index: []


Upload the cleaned DataFrame to the local database.

In [17]:
# Now let's initialise the engine that we will upload our DataFrame to

# We will set the 'yaml_file_path' to ensure the connection is made to the upload database engine
upload_engine = database.init_db_engine(yaml_file_path='rds_upload_db_creds.yaml')

# Upload 'cleaned_df' to our initialised engine
database.upload_to_db(cleaned_df, table_name='dim_users')

Called 'init_db_engine' method for database engine initialisation.
Read credentials from rds_upload_db_creds.yaml
Database credentials now stored in 'credentials' variable.
Database engine initialised successfully with the credentials in rds_upload_db_creds.yaml.
Data uploaded to the 'dim_users' table successfully.
